In [13]:
import numpy as np 
import pandas as pd 

In [14]:
dataset = pd.read_csv('../datasets/training.1600000.processed.noemoticon.csv',
                   sep=',',
                   header=None,
                   encoding='latin')
dataset.head(10)

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [15]:
dataset = dataset[[0, 5]]
# Estoy tomando las neutrales como positivas
dataset[0] = dataset[0].apply(lambda x:
                             0 if x == 0 else 1)
dataset.columns = ['sent', 'tweet']
dataset.head(10)

,sent,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
5,0,@Kwesidei not the whole crew
6,0,Need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,@Tatiana_K nope they didn't have it
9,0,@twittera que me muera ?


In [16]:
import re

hashtags = re.compile(r"^#\S+|\s#\S+")
mentions = re.compile(r"^@\S+|\s@\S+")
urls1 = r"(?:\@|https?\://)\S+"
urls2 = r"(?:\@|http?\://)\S+"
urls_pat = r'|'.join((urls1, urls2))
urls = re.compile(urls_pat)

def process_text(text):
  text = hashtags.sub(' hashtag', text)
  text = mentions.sub(' entity', text)
  return text.strip().lower()
  
def match_expr(pattern, string):
  return not pattern.search(string) == None

def get_data_wo_urls(dataset):
    link_with_urls = dataset.tweet.apply(lambda x: match_expr(urls, x))
    return dataset[[not e for e in link_with_urls]]

In [17]:
dataset.tweet = dataset.tweet.apply(process_text)
dataset.head(4)

,sent,tweet
0,0,"entity http://twitpic.com/2y1zl - awww, that's..."
1,0,is upset that he can't update his facebook by ...
2,0,entity i dived many times for the ball. manage...
3,0,my whole body feels itchy and like its on fire


In [18]:
from sklearn.model_selection import train_test_split
TRAIN_SIZE = 0.75
VAL_SIZE = 0.05
dataset_count = len(dataset)

df_train, df_test = train_test_split(dataset, test_size=1-TRAIN_SIZE, random_state=42)
# df_train_val, df_test = train_test_split(dataset, test_size=1-TRAIN_SIZE-VAL_SIZE, random_state=42)
# df_train, df_val = train_test_split(df_train_val, test_size=VAL_SIZE / (VAL_SIZE + TRAIN_SIZE), random_state=42)

print("TRAIN size:", len(df_train))
# print("VAL size:", len(df_val))
print("TEST size:", len(df_test))

TRAIN size: 1200000
TEST size: 400000


In [19]:
df_train = get_data_wo_urls(df_train)
df_train.head(4)

,sent,tweet
66270,0,entity dame!!!1
428045,0,entity thesaurus world sale ended
1307928,1,entity ight i let they lil white boy know. hah...
840793,1,entity you are loved


In [20]:
import time
import os
from joblib import Parallel, delayed

train_len = len(df_train.index)
texts_train = df_train['tweet'].tolist()
classes_train = df_train['sent'].tolist()
ids_train = [n for n in range(train_len)]

test_len = len(df_test.index)
texts_test = df_test['tweet'].tolist()
classes_test = df_test['sent'].tolist()
ids_test = [n for n in range(test_len)]

In [21]:
# def create_directory(directory):
#     try:
#         os.mkdir(directory)
#     except OSError:
#         print('OSError')
#     else:
#         pass

# path = os.getcwd()
# train_path = os.path.join(path, 'bert_twitter', 'train', '')
# test_path = os.path.join(path, 'bert_twitter', 'test', '')

# create_directory(os.path.join(os.getcwd(), 'bert_twitter'))
# create_directory(train_path)
# create_directory(test_path)

# create_directory(os.path.join(train_path, 'pos', ''))
# create_directory(os.path.join(train_path, 'neg', ''))
# create_directory(os.path.join(test_path, 'pos', ''))
# create_directory(os.path.join(test_path, 'neg', ''))

# def write_txt(text_, id_, class_, i, is_train):
#     if class_ == 0:
#         class_ = 'neg'
#     else:
#         class_ = 'pos'
#     if is_train == 0:
#         cur_path = test_path
#     else:
#         cur_path = train_path
#     with open(os.path.join(cur_path, class_, f'{i}_{class_}_{id_}.txt'), 'w', encoding='utf-8') as f:
#         f.write(text_)

# threads=12
# Parallel(n_jobs=threads)(delayed(write_txt)(texts_train[i], ids_train[i], classes_train[i], i, 1) for i in range(train_len))
# Parallel(n_jobs=threads)(delayed(write_txt)(texts_test[i], ids_test[i], classes_test[i], i, 0) for i in range(test_len))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [1]:
import ktrain
from ktrain import text
import os

using Keras version: 2.2.4-tf


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #To ignore GPU

In [3]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder("../datasets/bert_twitter",
                                                                        maxlen=500,
                                                                        preprocess_mode="bert",
                                                                        classes=["pos", "neg"])

detected encoding: utf-8
preprocessing train...
language: en


preprocessing test...
language: en


In [5]:
learner = ktrain.get_learner(text.text_classifier("bert", (x_train, y_train), preproc=preproc),
                            train_data = (x_train, y_train),
                            val_data = (x_test, y_test),
                            batch_size = 6)

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 1145142 samples, validate on 400000 samples
   2658/1145142 [..............................] - ETA: 685:17:56 - loss: 0.6804 - accuracy: 0.6068

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
data = ['Idiot.',
        'I like that.',
        'Where are you from?.',
        'What a beautiful romantic comedy. 10/10 would see again!',
        ]
predictor.predict(data)